In [ ]:
project = 'saga-trafikkdata-prod-pz8l'
use_colab_auth = True

# Legg inn ditt eget prosjekt her, f.eks. 'saga-olanor-playground-ab12'
bq_job_project = ''

In [ ]:
if (use_colab_auth):
  from google.colab import auth
  auth.authenticate_user()
  print('Authenticated')

In [ ]:
import warnings
from google.cloud import bigquery

warnings.filterwarnings('ignore')
client = bigquery.Client(project=bq_job_project)

Denne spørringen henter enkelt og greit ut timetrafikk for trafikkregistreringspunktet "HØVIK" på datoen 2. februar 2022, totalt for begge kjøreretninger.

In [ ]:
query = f"""
SELECT
    name AS tellepunkt,
    EXTRACT(DATE FROM `from` AT TIME ZONE "Europe/Oslo") AS dato,
    EXTRACT(HOUR FROM `from` AT TIME ZONE "Europe/Oslo") AS time,
    total.volumeNumbers.volume as timetrafikk
FROM `{project}.standardized.timetrafikk`
WHERE name = "HØVIK"
AND DATE(`from`, "Europe/Oslo") = "2022-02-02"
ORDER BY dato, time
""" 

print(query)

client.query(query).to_dataframe()

Følgende spørring henter data for tellepunktet på E18 ved Høvik og beregner gjennomsnittlig døgntrafikk i januar i år per ukedag, kjøreretning og lengdeklasse.

In [ ]:
query = f"""
SELECT
    name AS tellepunkt,
    flat_directions.heading AS retning,
    EXTRACT(DAYOFWEEK FROM `from` AT TIME ZONE "Europe/Oslo") AS ukedag,
    CAST(AVG(IF(flattened.lengthRange.upperBound = 5.6, flattened.total.volumeNumbers.volume, NULL)) * 24 AS INT64) AS korte_kjoretoy,
    CAST(AVG(IF(flattened.lengthRange.upperBound IS NULL AND flattened.lengthRange.lowerBound = 5.6, flattened.total.volumeNumbers.volume, NULL)) * 24 AS INT64) AS lange_kjoretoy
FROM `{project}.standardized.timetrafikk`, UNNEST(byDirection) flat_directions, UNNEST(flat_directions.byLengthRange) flattened
WHERE name = "HØVIK"
AND DATE(`from`, "Europe/Oslo") BETWEEN "2022-01-01" AND "2022-01-31"
AND (flattened.lengthRange.upperBound = 5.6 OR (flattened.lengthRange.lowerBound = 5.6 AND flattened.lengthRange.upperBound IS NULL))
GROUP BY 1,2,3
ORDER BY 1,2,3
""" 

print(query)

client.query(query).to_dataframe()

Denne spørringen viser en enkel `UNNEST` for å finne totalvolum og volum per __felt__.

In [ ]:
query = f"""
SELECT 
  trpId,
  DATETIME(`from`, "Europe/Oslo") AS dato,
  timetrafikk.total.volumeNumbers.volume as totalVolum,
  lanes.lane.laneNumber as felt,
  lanes.total.volumeNumbers.volume as feltVolum
FROM
  `{project}.standardized.timetrafikk` timetrafikk,
  UNNEST(byLane) lanes
WHERE
  DATE(`from`, "Europe/Oslo") = "2021-09-15"
  AND trpId = "16219V72812"
ORDER BY `from`
LIMIT 20
""" 

print(query)

client.query(query).to_dataframe()

Denne spørringen viser en enkel `UNNEST` for å finne totalvolum og volum per __lengdeklasse__.

In [ ]:
query = f"""
SELECT 
  trpId,
  DATETIME(`from`, "Europe/Oslo") AS dato,
  timetrafikk.total.volumeNumbers.volume as totalVolum,
  IFNULL(lengthRanges.lengthRange.lowerBound, 0) meterLengdeFra, 
  IFNULL(CAST(lengthRanges.lengthRange.upperBound AS STRING), "ubegrenset") meterLengdeTil,
  lengthRanges.total.volumeNumbers.volume as lengdeklasseVolum
FROM
  `{project}.standardized.timetrafikk` timetrafikk,
  UNNEST(byLengthRange) lengthRanges
WHERE
  DATE(`from`, "Europe/Oslo") = "2021-09-15"
  AND trpId = "16219V72812"
ORDER BY `from`
LIMIT 20
""" 

print(query)

client.query(query).to_dataframe()